# Analiza vpliva centralizacije na občine v Sloveniji

Cilj naloge je ugotoviti katere občine bi pridobile največ z decentralizacijo delovnih mest v Sloveniji. V nalogi bom analiziral migracije med Slovenskimi občinami in se naslajal na podatke o zaslužkih ljudi v danih občinah, njihovih migracijah ter cen nepremičnin. S temi podatki bom ugotavljal v katerih občinah bi bilo mogoče najbolj izboljšat zaslužek in standard življenja. 

## Uvoz podatkov

Na začetku naloge bom iz zbranih podatkov izračunal kazalnike, ki vplivajo na posamezno občino. S tem namenom naložimo vse podatke, ki jih bomo pri analizi uporabili.

In [1]:
import pandas as pd

# Nastavitve prikaza
pd.options.display.max_rows = 20

%matplotlib inline

# Podatki
import os.path

PODATKI = "./podatki"

nepremicnine = pd.read_csv(os.path.join(PODATKI, "nepremicnine.csv"), index_col="id")
podjetja = pd.read_csv(os.path.join(PODATKI, "podjetja.csv"), index_col="id")
razdalje = pd.read_csv(os.path.join(PODATKI, "razdalje.csv"), index_col="zacetek")
migracije_v_obcino = pd.read_csv(os.path.join(PODATKI, "migracije_v_obcino.csv"), index_col=[0])
migracije_iz_obcine = pd.read_csv(os.path.join(PODATKI, "migracije_iz_obcine.csv"), index_col=[0])
obcine = pd.read_csv(os.path.join(PODATKI, "obcine.csv"), index_col="ime")

# Pomožne vrednosti
vektor_obcin = pd.Series(obcine.index, index=obcine.index)

Poglejmo najprej s kakšnimi podatki bomo delali. Naložili smo podatke o nepremičninah, ki vsebujejo podatke o vrsti nepremičnine, ceni, vrsti posredovanja, številu sob in podobno. Prav tako sem zbral podatke o razdaljah med posameznimi občinami in migracijah, ki nam bodo pomagale ocenit koliko povprečno posameznik izgubi časa in denarja zaradi potrebnih poti.

Poleg teh podatkov, imamo še informacije o večjih Slovenskih podjetjih, kar nam bo pomagalo ocenit pri katera podjetja delajo največjo izgubo s trenutno ureditvijo.

Vse podatke pa bomo združevali po občinah, zato imamo tudi statistične podatke o posamezni občini.

## Obdelava podatkov o nepremičninah

Najprej izračunajmo povprečno vrednosti nepremičnin v posamezni občini. Pri tem se osredotočimo na hiše in stanovanja, ki se prodajajo in si zabeležimo še koliko nepremičnin prištejemo k povprečju.

Cene nepremičnin nam bodo pomagale ocenit, če je razlika v ceni nepremičnine, glede na to kam ljudje iz te občine migrirajo, smiselna glede na čas in stroške poti. V idealni situaciji bi živel tam, kjer je cena nepremičnin majhna, pot kratka, zaslužek pa velik.

In [2]:
# Vsem nepremičninam izračunamo ceno kvadratnega metra.
nepremicnine["cena_na_m2"] = nepremicnine.cena / nepremicnine.velikost

# Zajem
nepremicnine_po_obcinah = nepremicnine[
    nepremicnine.vrsta.isin(["Hiša", "Stanovanje"]) & 
    (nepremicnine.posredovanje == "prodaja")
].groupby("obcina")
    
# Obdelaj in priključi podatke k občinam.
obcine["cena_na_m2"] = nepremicnine_po_obcinah["cena_na_m2"].mean()
obcine["stevilo_nepremicnin"] = nepremicnine_po_obcinah.size()

## Obdelava podatkov o migracijah

Izračunajmo sedaj kako se premikajo migracije po Slovenskih regijah. Zanima nas, kakšen delež migrantov iz posamezne regije gre povprečno v dano drugo regijo. Povedano drugače, zanima nas kakšna je verjetnost, da oseba, ki živi v Ljubljani in migrira, migrira v Maribor, Koper...

To bomo izračunali na sledeč način:

1. Najprej bomo izračunali absolutno število ljudi, ki vsak dan migrira v Ljubljano. Ta podatek dobimo kot razliko delavno aktivnih prebivalcev, ki delajo v Ljubljani (po delovnem mestu) in delavno aktivnih prebivalcev, ki živijo v Ljubljani (po prebivališču), pomnožen z deležem lokalnih delavcev.

1. Glede na izbrano občino bomo izračunali kakšna je verjetnost, da gre ta oseba v druge občine. To bomo določili kot relativno število migracije v posamezno občino glede na vse ostale občine razen te, iz katere oseba prihaja. Ker ljudje raje delajo bližje doma, bom vrednosti obtežil glede na oddaljenost od dane občine.


In [3]:
# Izračunamo koliko časa in kilometrov v povprečju posameznik iz dane občine porabi.
def pot(obcina):
    migracije = migracije_iz_obcine[obcina]
    poti = razdalje[razdalje.index == obcina].set_index("konec")

    df = pd.DataFrame({ 
        "razdalja": poti.razdalja, 
        "cas": poti.cas, 
        "migracije": migracije
    }, index=migracije.index)

    df["rel_razdalja"] = df.razdalja * df.migracije
    df["rel_cas"] = df.cas * df.migracije
    
    return pd.Series(
        [
            df["rel_razdalja"].sum(), 
            df["rel_cas"].sum()
        ], 
        index=["razdalja", "cas"]
    )


# Izračunaj na vseh občinah.
poti = pd.DataFrame(vektor_obcin.apply(pot), columns=["razdalja", "cas"])

# Dodaj k občinam
obcine["razdalja_migracije"] = poti.razdalja
obcine["cas_migracije"] = poti.cas


# Izpis podatkov.
# poti.sort_values("razdalja").tail(20)
# poti["Grosuplje"], poti["Maribor"]
# migracije_iz_obcine["Grosuplje"].sort_values(ascending=False)

Na podlagi podatkov o migracijah lahko ocenimo koliko denarja posamezna občina nameni migracijam. To bom naredil tako, da bom na podlagi povprečne dolžine poti do delovnega mesta, izračunal koliko bencina porabijo ljudje za pot ter iz cene naftnih derivatov izračunal oceno potnih stroškov. Na podlagi povprečne neto plače v občini bom izračunal urno postavko delavca ter upošteval še povprečni čas vožnje na delovno mesto.

Povprečno porabo avtomobila sem pobral iz [te](https://www.iea.org/reports/fuel-consumption-of-cars-and-vans) strani in znaša malo več kot 7L/100km, kar bom zaokrožil na 7L/100km. Za ceno 1L goriva bom vzel kar 1€, saj je toliko znašala cena goriva v zadnjem letu.

Izračunal bom še znesek nadomestila, ki ga migrantje prejmejo od svojih podjetji kot obračun prevoza na delo. Povrnjen znesek iz [vira](https://mladipodjetnik.si/novice-in-dogodki/novice/obracun-prevoz-na-delo-ter-kilometrine-2) znaša 0.18€/km.

In [9]:
# Stroški prevoza

poraba_goriva = 7 / 100
cena_goriva = 1.0 / 1

delovni_dnevi_v_mescu = 4 * 5
st_potovanj = delovni_dnevi_v_mescu * 2

# Izračuni vrednosti

obcine["stroski_goriva"] = (obcine.razdalja_migracije / 1000) * poraba_goriva * cena_goriva * st_potovanj

# Čas potovanja
delovne_ure_v_mescu = delovni_dnevi_v_mescu * 8
obcine["povprecna_urna_postavka"] = obcine.placa_neto / delovne_ure_v_mescu

obcine["stroski_prevoza"] = (obcine.cas_migracije / 3600) * st_potovanj * obcine.povprecna_urna_postavka

# Povračilo potnih stroškov na prebivalca.
obcine["povracilo_stroskov_migracije_delavca"] = 0.18/1 * (obcine.razdalja_migracije / 1000) * st_potovanj

# Združimo vse stroške.
obcine["stroski_migracije_delavca"] = obcine.stroski_prevoza + obcine.stroski_goriva
obcine["razlika_v_stroskih_delavca"] = obcine.stroski_migracije_delavca - obcine.povracilo_stroskov_migracije_delavca


# Prikažemo rezultate
stroski_migracije = [
    "stroski_migracije_delavca",
    "razlika_v_stroskih_delavca",
    "povracilo_stroskov_migracije_delavca"
]

obcine.sort_values("stroski_migracije_delavca", ascending=False)[stroski_migracije]

,stroski_migracije_delavca,razlika_v_stroskih_delavca,povracilo_stroskov_migracije_delavca
ime,,,
Bovec,888.279015,-107.497515,995.776530
Kobarid,780.644483,-35.982025,816.626507
Tolmin,727.499435,2.456561,725.042874
Kanal,723.524060,-102.689778,826.213838
Brda,712.280618,-112.664134,824.944752
...,...,...,...
Domžale,245.406711,-11.102515,256.509226
Škofljica,240.791520,-32.997484,273.789004
Trzin,239.279957,-7.696247,246.976204


## Obdelava podatkov o podjetjih

Ocenili bi radi kakšno izgubo delajo podjetja zaradi plačevanja potnih stroškov svojih delavcev. To bomo naredili tako, da bomo za vsako podjetje ocenili koliko stroškov plača na enega zaposlenega glede na to v kateri občini je. Ta podatek bomo izračunali kot povprečje stroškov delavca iz posamezne občine glede na verjetnost, da delavec prihaja iz poljubne občine. 

Predpostavil bom, da imajo podjejta vsa delovna mesta v občini registracije in ocenil katera podjetja zaradi prevoza svojih delavcev plačajo največ potnih stroškov.

In [8]:
# Nastavitve parametrov

ODDALJENOST = 50_000 # koliko je spodnja meja, da začne podjetje plačevati potne stroške

# Stroški prevoza delavca za podjetje v posamezni občini.
def potni_stroski(obcina):
    """
    Vrne informacijo o potnih stroških posameznika, ki pride delat
    v dano občino.
    Za boljši približek, občine, ki so blizu naši občini filtriramo.
    """
    
    poti = razdalje[razdalje.index == obcina].set_index("konec")
    index_daljnih_obcin = poti[poti.razdalja > ODDALJENOST].index
    
    daljne_obcine = obcine[obcine.index.isin(index_daljnih_obcin)]
    stroski = migracije_v_obcino[obcina] * daljne_obcine.povracilo_stroskov_migracije_delavca
    
    return stroski.sum()

# Stroški podjetja za prevoz delavcev.
podjetja["potni_stroski"] = podjetja.obcina.apply(potni_stroski) * podjetja.st_zaposlenih

## Vrednost nepremičnine posameznemu delavcu

V tem delu raziskave želim izračunati koliko migrantje v posamezni občini prišparajo z nakupom nepremičnine zunaj občine v kateri delajo. Za izračun teh podatkov bom uporabil podatke o povprečni ceni nepremičnin v dani občini in verjetnosti, da občan dane občine v poljubni občini dela. Ker želim izračunati relativno ceno nepremičnine glede na posamezno občino bom gledal razliko v ceni nepremičnine med občino v kateri smo in občino v kateri migrant dela.

Na koncu tega dela raziskave bomo tako imeli idejo o tem v katerih občinah se najbolj splača kupiti hišo za delo in koliko okvirno z nakupom delavci prišparajo.

In [37]:
def relativna_cena_na_m2(obcina):
    """Vrne relativno razliko cene nepremičnin v občini glede na druge občine."""
    
    relativne_cene = (obcine.loc[obcina].cena_na_m2 - obcine.cena_na_m2) * migracije_iz_obcine[obcina]
    return relativne_cene.sum()

obcine["relativna_cena_na_m2"] = vektor_obcin.apply(relativna_cena_na_m2)

# Izpis vrednosti.
# obcine.sort_values("relativna_cena_na_m2", ascending=False).head(10)[["stevilo_nepremicnin", "cena_na_m2", "relativna_cena_na_m2"]]

,stevilo_nepremicnin,cena_na_m2,relativna_cena_na_m2
ime,,,
Dravograd,426.0,7753.638901,6031.240795
Ljubljana,1866.0,3065.210486,1553.384156
Grad,696.0,2738.354799,715.143931
Cankova,22.0,2079.016122,710.198322
Krško,329.0,2395.885211,573.604676
Horjul,6.0,2730.994301,518.464012
Bloke,107.0,2794.164225,513.433684
Hodoš/Hodos,59.0,1714.280349,384.102944
Metlika,477.0,2575.292028,374.493326


## Združenje podatkov o nepremičninah, podjetjih in migracijah.

V raziskavi smo sedaj izračunali okvirne vsote, ki jih podjetja porabijo za plačilo potnih stroškov svojih zaposlenih, izračunali smo koliko več ali manj v povprečju prebivalci dane občine porabijo za nakup nepremičnine v drugi občini kot v tisti v kateri delajo. Dodatno smo še izračunali povprečne stroške, ki bi jih morali migrantje zaradi vožnje na delo sami plačati, vendar jim jih plačajo podjetja kot potne stroške.

V zadnjem delu naloge bi radi ugotovili katere občine imajo največji potencial za decentralizacijo ter katera podjetja bodo pri tem najbolj pomagala. Ker največ ljudi migrira v večja mesta kot sta Maribor in Ljubljana bom raziskavo zožil na vprašanje o tem, v katero občino bi moral postavit coworking prostore, da bi ljudje prišparali največ časa.

Konkretizirajmo vprašanji:

1. Zanima me v katero občino se najbolj splača preselit, če bodo 